# DB2-Salesforce connector: Tool usage clusters mapping to tools

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'ToolToolClusterAssociation__c'

from nanoHUB.pipeline.application import Application

application = Application.get_instance()
wang159_myrmekes_db = application.new_db_engine('wang159_myrmekes')


salesforce = application.new_salesforce_engine()
db_s = salesforce


In [2]:
import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI

## Obtain tool information from DB2 

In [3]:

classtool_info_df = pd.read_sql_query('select * from cluster_classtool_info', wang159_myrmekes_db)

In [4]:
classtool_info_df.head(3)

,index,toolname,class_id
0,0,geneticalgo,0
1,1,geneticalgo,1
2,2,geneticalgo,2


## Obtain Salesforce IDs

In [6]:
# query the Salesforce IDs for contacts and citations. when updating junction objects, these IDs must be used

# get Salesforce ID for contacts
sf_toolname_df = db_s.query_data('SELECT Id, Tool_name__c FROM nanoHUB_tools__c where Tool_name__c != NULL')

[Success] Bulk job creation successful. Job ID = 7505w00000TmOs9AAF
{"id":"7505w00000TmOs9AAF","operation":"query","object":"nanoHUB_tools__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T17:09:20.000+0000","systemModstamp":"2021-01-21T17:09:20.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000TmOs9AAF","operation":"query","object":"nanoHUB_tools__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T17:09:20.000+0000","systemModstamp":"2021-01-21T17:09:20.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000TmOs9AAF","operation":"query","object":"nanoHUB_tools__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T17:0

In [7]:
# get Salesforce ID for citations
sf_clusterID_df = db_s.query_data('SELECT Id, ID__c FROM tool_usage_cluster__c')

sf_clusterID_df['ID__c'] = sf_clusterID_df['ID__c'].astype('int')

[Success] Bulk job creation successful. Job ID = 7505w00000TmOsOAAV
{"id":"7505w00000TmOsOAAV","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T17:09:53.000+0000","systemModstamp":"2021-01-21T17:09:53.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000TmOsOAAV","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T17:09:53.000+0000","systemModstamp":"2021-01-21T17:09:53.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000TmOsOAAV","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"

## Match data with Salesforce format

In [8]:
# Display
display(classtool_info_df.head(2))
display(sf_toolname_df.head(2))
display(sf_clusterID_df.head(2))

,index,toolname,class_id
0,0,geneticalgo,0
1,1,geneticalgo,1


,Id,Tool_name__c
0,a0s5w00000k5MOwAAM,ellipsom
1,a0s5w00000k5MOxAAM,hydrolab


,ID__c,Id
0,4,a0w5w000009Q7CuAAK
1,5,a0w5w000009Q7CvAAK


In [9]:
# merge dataframes
ct_tolink_df = pd.merge(classtool_info_df, sf_toolname_df, how='inner', left_on='toolname', right_on='Tool_name__c')\
                           .rename(columns={'Id':'SF_ID_tool'})

ct_tolink_df = pd.merge(ct_tolink_df, sf_clusterID_df, how='inner', left_on='class_id', right_on='ID__c')\
                           .rename(columns={'Id':'SF_ID_cluster'})

# display
ct_tolink_df.head(2)

,index,toolname,class_id,SF_ID_tool,Tool_name__c,ID__c,SF_ID_cluster
0,0,geneticalgo,0,a0s5w00000k5MXTAA2,geneticalgo,0,a0w5w00000A8U1UAAV
1,1,geneticalgo,1,a0s5w00000k5MXTAA2,geneticalgo,1,a0w5w00000A8OFeAAN


In [10]:
# create a new column for object name
ct_tolink_df['Name'] = ct_tolink_df.apply(lambda x: '%s_%d'%(x.Tool_name__c, x.ID__c), axis=1)

In [11]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name']         = ct_tolink_df['Name']
df_sf['Tool__c'] = ct_tolink_df['SF_ID_tool']
df_sf['Tool_Usage_Cluster__c'] = ct_tolink_df['SF_ID_cluster']

sf_original_fields = df_sf.columns

# display
display(df_sf.head(2).T)
display('Total rows: ', df_sf.shape[0])

,0,1
Name,geneticalgo_0,geneticalgo_1
Tool__c,a0s5w00000k5MXTAA2,a0s5w00000k5MXTAA2
Tool_Usage_Cluster__c,a0w5w00000A8U1UAAV,a0w5w00000A8OFeAAN


'Total rows: '

6002

## To Salesforce Sales Cloud CRM

In [12]:
# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [13]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000TmOsiAAF
hello
[Success] CSV upload successful. Job ID = 7505w00000TmOsiAAF
[Success] Closing job successful. Job ID = 7505w00000TmOsiAAF


In [14]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

{'apexProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apiVersion': 47.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-21T17:10:37.000+0000',
 'externalIdFieldName': 'Name',
 'id': '7505w00000TmOsiAAF',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 0,
 'object': 'ToolToolClusterAssociation__c',
 'operation': 'upsert',
 'retries': 0,
 'state': 'UploadComplete',
 'systemModstamp': '2021-01-21T17:10:37.000+0000',
 'totalProcessingTime': 0}
